In [1]:
## Import relevant libraries
import sys
import glob

sys.path.append('../../py_files/')
import quadrop as qd

qd.set_plotting_style()

### Data pre-procesing

In [4]:
# Example usage
base_dir = '../../../../Thomson Lab Dropbox/David Larios/activedrops/ubuntu/thtr-kif5-RT-temp'
qd.consolidate_images(base_dir)

Consolidation appears to be already done. Directory '../../../../Thomson Lab Dropbox/David Larios/activedrops/ubuntu/091024-sustainedMotors-RT/2p5TMB-1ulDNA100nM_' already exists with subfolders.


In [3]:
# Example usage
data_path = "../../../../Thomson Lab Dropbox/David Larios/activedrops/ubuntu/thtr-kif5-RT-temp/2p5TMB-1ulDNA100nM_/"

calibration_curve_paths = sorted(glob.glob("../../../../Thomson Lab Dropbox/David Larios/activedrops/calibration_curve/***ugml.tif"))


conditions_dict = {
    "Kif5-RT": "Pos5", 
    "ThTr-RT": "Pos7",
    # "negative": "Pos8"
}

# Organize PosX folders into condition folders
qd.organize_conditions(data_path, conditions_dict)

# Now run the existing functions to reorganize the tiffs and rename the folders
conditions, subconditions = qd.prepare_conditions(data_path)
time_interval_list = [45] * len(conditions)  # time intervals in seconds between frames for each condition
subconditions = ["Rep1"]
print("Conditions:", conditions)
print("Subconditions:", subconditions)

Conditions: ['Kif5-RT', 'ThTr-RT']
Subconditions: ['Rep1']


In [ ]:
qd.reorgTiffsToOriginal(data_path, conditions, subconditions)


### Generate movies

In [ ]:
# Call the function
qd.fluorescence_heatmap(
    data_path, 
    conditions, 
    subconditions, 
    channel='cy5', 
    time_interval_list=time_interval_list, 
    vmax=14, 
    skip_frames=1, 
    calibration_curve_paths=calibration_curve_paths, 
    show_scalebar=False,
    )

In [ ]:
# Example usage
qd.create_movies(
    data_path, 
    conditions, 
    subconditions, 
    channel='cy5', 
    frame_rate=120,
    skip_frames=1
    )


In [ ]:
qd.create_combined_heatmap_movie_custom_grid(
    data_path, 
    conditions, 
    subconditions, 
    channel='cy5', 
    grid_rows=1, 
    grid_cols=2, 
    frame_rate=120,
    batch_size=50
    )


In [ ]:
# Call the function
qd.fluorescence_heatmap(
    data_path, 
    conditions, 
    subconditions, 
    channel='GFP', 
    time_interval_list=time_interval_list, 
    vmax=500, 
    skip_frames=1, 
    calibration_curve_paths=calibration_curve_paths, 
    show_scalebar=True,
    )

In [ ]:
# Example usage
qd.create_movies(
    data_path, 
    conditions, 
    subconditions, 
    channel='GFP', 
    frame_rate=120,
    skip_frames=1
    )


In [ ]:
qd.create_combined_heatmap_movie_custom_grid(
    data_path, 
    conditions[:-1], 
    subconditions, 
    channel='GFP', 
    grid_rows=2, 
    grid_cols=4, 
    frame_rate=120,
    batch_size=50
    )


In [ ]:
# qd.delete_temporary_image_directories(data_path, conditions, subconditions)

### Fluorescence Quantification

In [ ]:
# Example usage
mw_kda_list = [50, 50] 
droplet_volume_list = [2] * len(conditions)
protein_lengths_list = [500, 500] 

qd.quantify_tiffiles(
    data_path, 
    conditions, 
    subconditions, 
    calibration_curve_paths, 
    mw_kda_list, 
    droplet_volume_list, 
    time_interval_list, 
    protein_lengths_list,
    ribosome_count=10**9,
    skip_frames=1,
    subtract_negative=True
)

### PIV pre-processing

In [ ]:
qd.split_tiffs(data_path, conditions[1:], subconditions, channel='dapi', file_interval=1)

### PIV

In [4]:
# Define feature limits and other parameters
v = 2E-7
velocity_limits = (0, v)
other_limits = (-0.0005, 0.0005)
skip_frames = 1 ### CHANGE THIS TO SKIP FRAMES


velocity_limits = (None, None)
other_limits = (None, None)


feature_limits = {
    # 'u [m/s]': (-v, v), 
    # 'v [m/s]': (-v, v), 
    # 'data type [-]': (None, None),
    'velocity magnitude [m/s]': velocity_limits,
    'vorticity [1/s]': other_limits,
    'divergence [1/s]': other_limits,
    # 'dcev [1]': (0, 250),
    'shear [1/s]': other_limits,
    'strain [1/s]': other_limits,
    'vector direction [degrees]': (-180, 180),
}


# Features for PCA and plotting
features_pca = [
    "vorticity [1/s]_mean",
    "velocity magnitude [um/s]",
    "distance [m]_mean",
    "divergence [1/s]_mean",
    "shear [1/s]_mean",
    "strain [1/s]_mean",
    "correlation length [um]", 
    "power [W]_mean",
    "work [J]",
]


In [ ]:
time_interval_list = [45*3, 45]   # time intervals in seconds between frames for each condition


# Process PIV data
qd.process_piv_data(
    data_path, 
    conditions,
    subconditions, 
    feature_limits, 
    time_interval_list, 
    min_frame=0, 
    max_frame=None, 
    skip_frames=1, 
    plot_autocorrelation=False, 
    frame_rate=1, 
    heatmaps=False
    )


In [ ]:
# Plot features and PCA
qd.plot_PIV_all(
    data_path, 
    ['A-RT', 'C-RT', 'D-RT', 'E-RT', 'G-RT', 'H-RT'],
    subconditions, 
    features_pca, 
    min_frame=0, 
    max_frame=None
    )


### Expression + PIV

In [ ]:
# Combine the data and save it to the "output_data" directory
qd.combine_averaged_dataframes(data_path, conditions, subconditions)


In [ ]:
qd.merge_expression_piv_data(data_path)


In [ ]:
# # Specify the columns you want to plot
x_column = "Protein Concentration_nM"  # Example x-axis column
y_column = "velocity magnitude [m/s]_mean"  # Example y-axis column

 
# Apply Gaussian smoothing with sigma_x=2 and sigma_y=1
qd.plot_columns(data_path, x_column, y_column, sigma_x=1, sigma_y=5, x_log=True, y_log=False, min_frame=0, max_frame=None)


In [ ]:

# List of features for PCA
features_pca = [
    "vorticity [1/s]_mean",
    # "velocity magnitude [m/s]_mean",
    # "distance [m]_mean",
    "divergence [1/s]_mean",
    "shear [1/s]_mean",
    "strain [1/s]_mean",
    # "correlation length [m]_mean", 
    "power [W]_mean",
    # "work [J]_mean",
    'vector direction [degrees]_mean',
    "Protein Concentration_nM", 
]

# Run PCA and save plot (with all conditions and subconditions in the same plot)
qd.plot_pca_expression_piv(data_path, conditions=conditions, subconditions=subconditions, features=features_pca, sigma=1)


In [ ]:

# Example usage
qd.delete_outputs(data_path, conditions, subconditions, output_dirs=None)
